In [80]:
import glob
import numpy as np
from skimage import io
from skimage.external import tifffile
from skimage.transform import rescale
from scipy.ndimage import zoom
import time



In [81]:
search_directory = "C:/Data"
file_mask = '0.png'
chunk_names = glob.glob(search_directory + '/**/*' + file_mask,recursive=True)

In [82]:
x = (chunk_names[0:20])
print(x)

['C:/Data\\0\\1\\0\\0.png', 'C:/Data\\0\\10\\0\\0.png', 'C:/Data\\0\\100\\0\\0.png', 'C:/Data\\0\\1000\\0\\0.png', 'C:/Data\\0\\1001\\0\\0.png', 'C:/Data\\0\\1002\\0\\0.png', 'C:/Data\\0\\1003\\0\\0.png', 'C:/Data\\0\\1004\\0\\0.png', 'C:/Data\\0\\1005\\0\\0.png', 'C:/Data\\0\\1006\\0\\0.png', 'C:/Data\\0\\1007\\0\\0.png', 'C:/Data\\0\\1008\\0\\0.png', 'C:/Data\\0\\1009\\0\\0.png', 'C:/Data\\0\\101\\0\\0.png', 'C:/Data\\0\\1010\\0\\0.png', 'C:/Data\\0\\1011\\0\\0.png', 'C:/Data\\0\\1012\\0\\0.png', 'C:/Data\\0\\1013\\0\\0.png', 'C:/Data\\0\\1014\\0\\0.png', 'C:/Data\\0\\1015\\0\\0.png']


In [83]:
for fname in x:
    print(fname.split("\\")[-3])

1
10
100
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
101
1010
1011
1012
1013
1014
1015


In [85]:
XY_bin = 2
Z_bin = 8
a = [int(fname.split("\\")[-3]) for fname in chunk_names ]
zz = np.asarray([xx for (yy,xx) in sorted(zip(a,chunk_names))])
n_files = np.floor_divide(len(zz), Z_bin) * Z_bin
n_files = 40
print(zz[0:n_files].shape)
b = zz[0:n_files].reshape(-1,8)
print(b[0,:])

target_file_name = 'C:/data/temp.tif'

(40,)
['C:/Data\\0\\1\\0\\0.png' 'C:/Data\\0\\2\\0\\0.png'
 'C:/Data\\0\\3\\0\\0.png' 'C:/Data\\0\\4\\0\\0.png'
 'C:/Data\\0\\5\\0\\0.png' 'C:/Data\\0\\6\\0\\0.png'
 'C:/Data\\0\\7\\0\\0.png' 'C:/Data\\0\\8\\0\\0.png']


In [86]:

xc = 1200
dx = 2350
yc = 1050
dy = 1800
xi = xc - dx/2
xa = xi + dx

yi = yc - dy/2
ya = yi + dy

dz = n_files/Z_bin

indices = np.arange(dz)
frame_names_2D = b[0:n_files].reshape(-1,Z_bin)

In [40]:
t0 = time.time()
with tifffile.TiffWriter(target_file_name, bigtiff=True) as tif:
    for chunk_name, cnt in zip(frame_names_2D, indices):
        #print('Transferring data file:  ', chunk_name)
        new_frame = np.zeros((dy,dx,), dtype=float)
        for fr in chunk_name:
#             print(fr)
            new_frame+= io.imread(fr)[yi:ya,xi:xa]
        print(fr)
        new_frame = new_frame/4
        tif.save(new_frame.astype(int))
    tif.close()
t1 = time.time()
print(t1-t0)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


C:/Data\0\4\0\0.png
C:/Data\0\8\0\0.png
C:/Data\0\12\0\0.png
C:/Data\0\16\0\0.png
C:/Data\0\20\0\0.png
C:/Data\0\24\0\0.png
C:/Data\0\28\0\0.png
C:/Data\0\32\0\0.png
C:/Data\0\36\0\0.png
C:/Data\0\40\0\0.png
10.091000080108643


In [55]:
start = time.time()
with tifffile.TiffWriter(target_file_name, bigtiff=True) as tif:
    for chunk_names, cnt in zip(frame_names_2D, indices):
    # frame_subset = np.zeros((Z_bin, dy,dx,), dtype=float)
    # for fr in chunk_name:
    # new_frame+= io.imread(fr)[yi:ya,xi:xa]
#         print(chunk_names)
        frame_subset = np.array([io.imread(fr)[yi:ya,xi:xa] for fr in chunk_names])
        # new_frame = new_frame/Z_bin
        new_frame = zoom(frame_subset,[1/Z_bin, 1/XY_bin, 1/XY_bin])
        tif.save(new_frame.astype(frame_subset.dtype))
        print('Transferring CHUNK:  ', cnt)
    tif.close()
end = time.time()
print(end - start)
print(new_frame.shape)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Program Files\Anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:571: RuntimeWarning: divide by zero encountered in true_divide
  zoom = (numpy.array(input.shape) - 1) / zoom_div


Transferring CHUNK:   0.0
Transferring CHUNK:   1.0
Transferring CHUNK:   2.0
Transferring CHUNK:   3.0
Transferring CHUNK:   4.0
Transferring CHUNK:   5.0
Transferring CHUNK:   6.0
Transferring CHUNK:   7.0
Transferring CHUNK:   8.0
Transferring CHUNK:   9.0
43.519999980926514
(1, 1800, 2350)


In [43]:
def bin_ndarray(ndarray, new_shape, operation='sum'):
    """
    Bins an ndarray in all axes based on the target shape, by summing or
        averaging.

    Number of output dimensions must match number of input dimensions and 
        new axes must divide old ones.

    Example
    -------
    >>> m = np.arange(0,100,1).reshape((10,10))
    >>> n = bin_ndarray(m, new_shape=(5,5), operation='sum')
    >>> print(n)

    [[ 22  30  38  46  54]
     [102 110 118 126 134]
     [182 190 198 206 214]
     [262 270 278 286 294]
     [342 350 358 366 374]]

    """
    operation = operation.lower()
    if not operation in ['sum', 'mean']:
        raise ValueError("Operation not supported.")
    if ndarray.ndim != len(new_shape):
        raise ValueError("Shape mismatch: {} -> {}".format(ndarray.shape,
                                                           new_shape))
    compression_pairs = [(d, c//d) for d,c in zip(new_shape,
                                                  ndarray.shape)]
    flattened = [l for p in compression_pairs for l in p]
    ndarray = ndarray.reshape(flattened)
    for i in range(len(new_shape)):
        op = getattr(ndarray, operation)
        ndarray = op(-1*(i+1))
    return ndarray

In [92]:
start = time.time()
nsh = (1,dy/XY_bin,dx/XY_bin)
print(nsh)
with tifffile.TiffWriter(target_file_name, bigtiff=True) as tif:
    for chunk_names, cnt in zip(frame_names_2D, indices):
        x = np.array([io.imread(fr)[yi:ya,xi:xa] for fr in chunk_names])
        frame_subset = bin_ndarray(np.asarray(x,dtype=np.float32), new_shape=nsh, operation='mean')
        tif.save(frame_subset.astype(x.dtype))
        print('Transferring CHUNK:  ', cnt)
    tif.close()
end = time.time()
print(end - start)

(1, 900.0, 1175.0)


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:31: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Transferring CHUNK:   0.0
Transferring CHUNK:   1.0
Transferring CHUNK:   2.0
Transferring CHUNK:   3.0
Transferring CHUNK:   4.0
11.225000143051147


In [88]:
tif.close()
